In [58]:
# some ape notebook examples:
# curve steth analysis: https://github.com/fmrmf/hello-ape/blob/main/console/hello-ape.ipynb
# uniswap usdc/usdt 1bps pool analysis: https://gist.github.com/banteg/dcf6082ff7fc6ad51ce220146f29d9ff

from ape import accounts, chain, Contract, networks
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [4]:
networks.parse_network_choice('ethereum:mainnet:alchemy').__enter__()

<alchemy chain_id=1>

In [74]:
seaport = Contract("0x00000000006c3852cbEf3e08E8dF289169EdE581")
seaport.name()

'Seaport'

In [6]:
seaport.OrderFulfilled

OrderFulfilled(bytes32 orderHash, address indexed offerer, address indexed zone, address recipient, (uint8,address,uint256,uint256)[] offer, (uint8,address,uint256,uint256,address)[] consideration)

In [7]:
curr_block = chain.blocks[-1].number
blocks_1d = int(60*60*24/12)

In [8]:
start_block = curr_block - blocks_1d
start_block

15457908

In [9]:
%time recent_trades = seaport.OrderFulfilled.query("*", start_block=start_block)

INFO: Cache database has not been initialized
CPU times: user 35 s, sys: 1.67 s, total: 36.7 s
Wall time: 37.6 s


In [10]:
recent_trades.head()

,event_name,contract_address,event_arguments,transaction_hash,block_number,block_hash,log_index,transaction_index
0,OrderFulfilled,0x00000000006c3852cbEf3e08E8dF289169EdE581,{'offerer': '0xa028a1e8b57b4baa77867b529650886...,0x267684f029d2c804ffda5deae0850baf9e37486aa622...,15457908,0x1efaa679ce4373843ea455ae9b840c8aa864289fabd9...,106,95
1,OrderFulfilled,0x00000000006c3852cbEf3e08E8dF289169EdE581,{'offerer': '0x197A17071753Cd298970412cb30E67E...,0x72384213e719ce93db4edc9ef3faf912d4fe6106e669...,15457908,0x1efaa679ce4373843ea455ae9b840c8aa864289fabd9...,115,107
2,OrderFulfilled,0x00000000006c3852cbEf3e08E8dF289169EdE581,{'offerer': '0xb1A495A26963b83C8fE024c03CAe8A9...,0x1b66885d5ca1538a0d5a1bc531580247a2594f15d7a5...,15457908,0x1efaa679ce4373843ea455ae9b840c8aa864289fabd9...,127,112
3,OrderFulfilled,0x00000000006c3852cbEf3e08E8dF289169EdE581,{'offerer': '0x331f9e988d9Eb82e7962c8c8f4b965B...,0x1b66885d5ca1538a0d5a1bc531580247a2594f15d7a5...,15457908,0x1efaa679ce4373843ea455ae9b840c8aa864289fabd9...,128,112
4,OrderFulfilled,0x00000000006c3852cbEf3e08E8dF289169EdE581,{'offerer': '0x8951F25aDcC7C3256dd9b521ED726fB...,0x3a8db23a564c819c215f32f79adaaedf60415b6d40bc...,15457908,0x1efaa679ce4373843ea455ae9b840c8aa864289fabd9...,146,115


In [11]:
recent_trades["event_arguments"].iloc[-1]

{'offerer': '0x5C13A95bB4130c7Cc5e7353DeF09b79a3c557eAf',
 'zone': '0x004C00500000aD104D7DBd00e3ae0A5C00560C00',
 'orderHash': HexBytes('0x6a8720154307af1a0e35085bba5ef9ebc33e928cc003f7a9925325f5675b7534'),
 'recipient': '0x026F36DEad1CC6c522a201dbDD7410421Bb45303',
 'offer': ((2, '0x68c7a55a0b79840ed064ba1ccc7bd3b4863ff8f0', 43, 1),),
 'consideration': ((0,
   '0x0000000000000000000000000000000000000000',
   0,
   26325000000000000,
   '0x5c13a95bb4130c7cc5e7353def09b79a3c557eaf'),
  (0,
   '0x0000000000000000000000000000000000000000',
   0,
   675000000000000,
   '0x0000a26b00c1f0df003000390027140000faa719'))}

In [106]:
"""
 *   event OrderFulfilled(
 *     bytes32 orderHash,
 *     address indexed offerer,
 *     address indexed zone,
 *     address fulfiller,
 *     SpentItem[] offer > (itemType, token, id, amount)
 *     ReceivedItem[] consideration > (itemType, token, id, amount, recipient)
 *   )
 *
"""

def parse_trades(trades):
    blocks = trades.iloc[::1000]["block_number"].to_list()
    timestamps = [chain.blocks[i].timestamp for i in blocks]
    trades["timestamp"] = pd.to_datetime(np.interp(trades["block_number"], blocks, timestamps) * 1e9)
    offers = [args["offer"] for args in recent_trades["event_arguments"]]
    receipts = [args["consideration"] for args in recent_trades["event_arguments"]]
    trades["offer_tokens"] = [offer[0][1] if len(offer) > 0 else 'null' for offer in offers]
    trades["receipt_tokens"] = [receipt[0][1] if len(receipt) > 0 else 'null' for receipt in receipts]
    return trades

In [107]:
%time parsed_trades = parse_trades(recent_trades)

CPU times: user 574 ms, sys: 27.2 ms, total: 601 ms
Wall time: 8.37 s


In [108]:
top_offered_tokens = pd.DataFrame(parsed_trades["offer_tokens"].value_counts().reset_index().values, 
                                  columns=["Offer token address", "Occurrences"])
top_received_tokens = pd.DataFrame(parsed_trades["receipt_tokens"].value_counts().reset_index().values,
                                  columns=["Receipt token address", "Occurrences"])


In [115]:
top_offered_tokens_contracts = [Contract(address) if address != 'null' else 'null' 
                                for address in top_offered_tokens["Offer token address"].head(5)]

In [114]:
top_received_tokens_contracts = [Contract(address) if address != 'null' 
                                and address != '0x0000000000000000000000000000000000000000'
                                else 'null'
                                for address in top_received_tokens["Receipt token address"].head(5)]

In [116]:
# most common offered token in seaport trades in the past day
top_offered_tokens_contracts

[<WETH9 0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2>,
 <Delusion 0x2c0d708C6E82DC6c427CE6A96BA5ff2EA09bE272>,
 <GangApeBlingClub 0xE0d2ACE5DEbC43d3e0e758577D111C4859DF6F7e>,
 <Namida 0x81cF764150665eE2F4FA62Ca1A9Ff4324b29ec2B>,
 <Fregfreg 0x676Fb773Bb66Fd1E34d32d2A56053FE341b84064>]

In [117]:
# most common received token in seaport trades in the past day
top_received_tokens_contracts

['null',
 'null',
 <BaseRegistrarImplementation 0x57f1887a8BF19b14fC0dF6Fd9B2acc9Af147eA85>,
 <RedeemableToken 0xa49a0e5eF83cF89Ac8aae182f22E6464B229eFC8>,
 <Gala 0x15D4c048F83bd7e37d49eA4C83a07267Ec4203dA>]